<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using SingleStore Kai with LangChain</h1>
    </div>
</div>

In [6]:
!pip cache purge --quiet

In [7]:
!pip install langchain-community --quiet
!pip install langchain-openai --quiet
!pip install openai --quiet

In [8]:
import pymongo

from langchain_core.documents import Document
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from singlestoredb.management import get_secret

In [9]:
kai_client = pymongo.MongoClient(connection_url_kai)
db = kai_client["langchain_demo"]
collection = db["langchain_docs"]

In [10]:
os.environ["OPENAI_API_KEY"] = get_secret("OPENAI_API_KEY")

In [11]:
documents = [
    "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
    "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
    "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
    "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
    "Llamas are vegetarians and have very efficient digestive systems",
    "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old"
]

embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small"
)

dimensions = len(embeddings.embed_query(documents[0]))

docs = [Document(text) for text in documents]

In [12]:
collection.create_index(
    [("embedding", "vector")],
    name = "vector_index",
    kaiIndexOptions = {
        "index_type": "AUTO",
        "metric_type": "DOT_PRODUCT",
        "dimensions": dimensions
    }
)

'vector_index'

In [13]:
docsearch = MongoDBAtlasVectorSearch.from_documents(
    docs,
    embeddings,
    collection = collection,
    index_name = "vector_index"
)

In [14]:
prompt = "What animals are llamas related to?"
docs = docsearch.similarity_search(prompt)
data = docs[0].page_content
print(data)

Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels


In [15]:
openai_client = OpenAI()

response = openai_client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Provide more details."},
        {"role": "user", "content": f"Using this data: {data}. Respond to this prompt: {prompt}"}
    ]
)
print(response.choices[0].message.content)

Llamas are closely related to several animals within the camelid family. They share a close relationship with vicuñas, which are wild relatives of llamas found in South America. Additionally, llamas are related to alpacas, another domesticated species of camelid. Furthermore, they are distantly related to camels, including both the dromedary (one-humped) and Bactrian (two-humped) camels. All these species belong to the family Camelidae, highlighting their shared ancestry and characteristics.


## Cleanup

In [16]:
collection.drop()

In [17]:
kai_client.drop_database(db)

In [18]:
kai_client.close()